In [25]:
import json
import math
import re
import numpy as np
import pandas as pd

In [26]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great-circle distance in meters between two points 
    on the Earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    
    # Radius of Earth in meters (mean radius = 6,371km)
    r = 6371000
    return c * r


In [27]:
def filter_by_radius(df, ref_lat, ref_lon, radius_km=5.0,max_speed = 10):
    """
    Filters DataFrame to rows within a specified radius (km) of a reference point.
    
    Args:
        df: DataFrame with 'latitude' and 'longitude' columns
        ref_lat: Reference latitude (center point)
        ref_lon: Reference longitude (center point)
        radius_km: Radius in kilometers (default: 5.0 km)
        
    Returns:
        Filtered DataFrame with added 'distance_km' column
    """
    # Calculate distance for each row
    low_speed_df = df[df['speed'] < max_speed].copy()
    
    # Calculate distance for remaining points
    low_speed_df['distance_km'] = low_speed_df.apply(
        lambda row: haversine(ref_lat, ref_lon, row['latitude'], row['longitude']),
        axis=1
    )
    
    # Filter rows within radius
    filtered_df = low_speed_df[low_speed_df['distance_km'] <= radius_km].copy()
    
    return filtered_df

In [28]:
caseid = '23608592' 
prefix = caseid + '/' + caseid

In [29]:
with open(prefix +'_validations.geojson', 'r') as f:
    geojson_data = json.load(f)
with open(prefix +'_full_topology_data.geojson', 'r') as f:
    topojson_data = json.load(f)
with open(prefix +'_signs.geojson', 'r') as f:
    signjson_data = json.load(f)

probedf = pd.read_csv(prefix +'_probe_data.csv')

In [30]:
def coordcheck(c1s,c2,distance_threshold):
    for c1 in c1s:
        distance = haversine(c1[0],c1[1],c2[0],c2[1])
        if distance < distance_threshold:
            return [True, [c1[0],c1[1]]]
    return [False, []]

In [31]:
for i in range(0,len(geojson_data['features'])):
    signid = geojson_data['features'][i]['properties']['Feature ID']
    ermes = geojson_data['features'][i]['properties']['Error Message']
    topo = re.search(r'urn:here::here:Topology:(\d+)', ermes).group(0)
    print(i)
    print(geojson_data['features'][i])
    for j in range(0,len(signjson_data['features'])):
        c2 = signjson_data['features'][j]['properties']['id']
        if signjson_data['features'][j]['properties']['signType'] == 'MOTORWAY' and signid == c2:
            existingscore = signjson_data['features'][j]['properties']['confidence']['simpleScores'][0]['existenceProbability']
            print(existingscore)
            observedict = signjson_data['features'][j]['properties']['observationCounts']
            observescore = observedict['NInputsObserved']/observedict['NInputsObservable']
            print(observescore)
            print(j)
    for x in range(0,len(topojson_data['features'])):
        c3 = topojson_data['features'][x]['properties']['id']
        if topo == c3:
            motorwaydict = topojson_data['features'][x]['properties']['topologyCharacteristics']['isMotorway'][0]
            asscoiatedtopo = topojson_data['features'][x]
            print(motorwaydict)
            print(x)

0
{'type': 'Feature', 'properties': {'Rule Code': 'WSIGN406', 'Feature ID': 'urn:here::here:signs:1622370100585006459', 'Feature Type': 'Road Sign', 'Error Message': 'Motorway Sign urn:here::here:signs:1622370100585006459 at Lat 10.20583389 Lon 53.47428404 is associated to a Topology urn:here::here:Topology:532975241 that has a range for Pedestrian = TRUE within 20m distance.', 'Status': 'ACTIVE', 'LE Allowed': False, 'Clean for Extract': False, 'Partition ID': 23608592, 'ISO': 'DEU', 'Violation ID': 'urn:here::here:Violation:signs-e7b37c12-93b8-3216-8270-b665256de56f', 'is_within': True}, 'geometry': {'type': 'Point', 'coordinates': [10.20583, 53.47428]}}
0.22321988742285842
0.9
360
{'range': {'endOffset': 1.0, 'startOffset': 0.0}, 'value': False}
1575


In [32]:
# def computefacing(quat):
#     w, x, y, z = quat['w'], quat['x'], quat['y'], quat['z']
    
#     # Calculate forward vector
#     forward = np.array([
#         2 * (x*z + w*y),      # East-West (X)
#         2 * (y*z - w*x),      # North-South (Y) 
#         1 - 2 * (x*x + y*y)   # Up-Down (Z)
#     ])
#     forward /= np.linalg.norm(forward)
    
#     # Calculate bearing (0°=North, 90°=East)
#     bearing = np.degrees(np.arctan2(forward[0], forward[1])) % 360
#     return bearin

In [33]:
signcord = geojson_data['features'][0]['geometry']['coordinates']
# face =  signjson_data['features'][0]['properties']['orientation']['orientation']

In [41]:
asscoiatedtopo

{'type': 'Feature',
 'properties': {'partition_id': '23608592',
  'id': 'urn:here::here:Topology:532975241',
  'confidence': {'simpleScores': [{'featureType': 'ROAD_USAGE_FEE_REQUIRED',
     'scoreType': 'EXISTENCE',
     'score': 0.0,
     'updateSource': 'INJECTION_TMOB',
     'updatedOn': '2021-02-17T21:44:27.863Z'},
    {'featureType': 'ADAS_CURVATURE',
     'scoreType': 'EXISTENCE',
     'score': 0.0,
     'updateSource': 'INJECTION_TMOB',
     'updatedOn': '2021-02-17T21:44:27.863Z'},
    {'featureType': 'ADAS_SLOPE',
     'scoreType': 'EXISTENCE',
     'score': 0.0,
     'updateSource': 'INJECTION_TMOB',
     'updatedOn': '2021-02-17T21:44:27.863Z'},
    {'featureType': 'ADAS_ELEVATION',
     'scoreType': 'EXISTENCE',
     'score': 0.0,
     'updateSource': 'INJECTION_TMOB',
     'updatedOn': '2021-02-17T21:44:27.863Z'}]},
  'zLevel': None,
  'adasTopology': {'startNodeTraversals': [{'references': [{'id': 'urn:here::here:Topology:208461017',
       'referencePoint': {'type': 'Po

In [ ]:
import matplotlib.pyplot as plt
from itertools import cycle
color_cycle = cycle(['red', 'green', 'blue', 'orange', 'purple', 'cyan'])
asscoiatecond, coord = coordcheck(asscoiatedtopo['geometry']['coordinates'],signcord,20)
if existingscore < 0.5 and observescore<0.5:
    print('sign do not exsit since both existingscore and observescore are below 0.5')
else:
    nearbytopo = []
    for i in range(0,len(topojson_data['features'])):
        condition, coord = coordcheck(topojson_data['features'][i]['geometry']['coordinates'],signcord,20)
        if condition:
            nearbytopo.append([topojson_data['features'][i],coord])

    for j in range(0,len(nearbytopo)):
        temptopo = nearbytopo[0]
        coordinates = temptopo[j]['geometry']['coordinates']
        print(temptopo[j]['properties']['accessCharacteristics'][0]['pedestrian'])
        #TODO
        # task one: using coord to compute which part is the sign fall into. return ismotorway true or false

        print(temptopo[j]['properties']['topologyCharacteristics']['isMotorway'])


        #TODO
        #if asscoiatedtopo ismotorway and pedestrian is True: we correct pedestrian to False



        #TODO
        #if asscoiatedtopo ismotorway is false: find nearby topo where ismotorway is True and pedestrian is false and we assgin it to the validation

        #TODO
        #if nothing is found. We check probe data and see if pedestrain nearby. if yes, case 4. Otherwise, we correct pedestrian to False



        # Unzip coordinates into separate X and Y lists
        x, y = zip(*coordinates)  # Unpack tuples into two lists

        # Plot
        plt.plot(x, y, 
                color=next(color_cycle),
                marker='o',      # Add markers at each point
                linestyle='-',   # Solid line
                linewidth=2,     # Thicker line
                markersize=5,    # Size of markers
                label='Path'+str(j))
    coordinates = asscoiatedtopo['geometry']['coordinates']
    x, y = zip(*coordinates)
    plt.plot(x, y, 
                    color='yellow', 
                    marker='o',      # Add markers at each point
                    linestyle='-',   # Solid line
                    linewidth=2,     # Thicker line
                    markersize=5,    # Size of markers
                    label='asscoiated')
    plt.plot(signcord[0],signcord[1],color='black',marker='o')
    # Customize the plot
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('Line Plot of Coordinates')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()
    plt.tight_layout()
    plt.show()

TypeError: list indices must be integers or slices, not str

In [53]:
print(nearbytopo[4]['properties']['topologyCharacteristics']['isMotorway'])

[{'range': {'endOffset': 0.12200490495218867, 'startOffset': 0.0}, 'value': False}, {'range': {'endOffset': 1.0, 'startOffset': 0.12200490495218867}, 'value': True}]


Below is the probe solution

In [ ]:
# for i in range(0,len(topojson_data['features'])):
#         condition, coord = coordcheck(topojson_data['features'][i]['geometry']['coordinates'],signcord,20)
#         if condition:
#             coordinates = topojson_data['features'][i]['geometry']['coordinates']
#             filtered_probedf= filter_by_radius(probedf,coord[1],coord[0],20,1)
#             print(coord)
#             print(filtered_probedf.shape)
#             if (filtered_probedf['speed'] < 10).any():
#                 print('not highway')
#             else:
#                 print('highway')
#             print(topojson_data['features'][i]['properties']['accessCharacteristics'][0]['pedestrian'])
#             # Unzip coordinates into separate X and Y lists
#             x, y = zip(*coordinates)  # Unpack tuples into two lists

#             # Plot
#             plt.plot(x, y, 
#                     color=next(color_cycle),
#                     marker='o',      # Add markers at each point
#                     linestyle='-',   # Solid line
#                     linewidth=2,     # Thicker line
#                     markersize=5,    # Size of markers
#                     label='Path')
#     coordinates = topojson_data['features'][676]['geometry']['coordinates']
#     x, y = zip(*coordinates)
#     # plt.plot(x, y, 
#     #                 color='yellow', 
#     #                 marker='o',      # Add markers at each point
#     #                 linestyle='-',   # Solid line
#     #                 linewidth=2,     # Thicker line
#     #                 markersize=5,    # Size of markers
#     #                 label='asscoiated')
#     plt.plot(signcord[0],signcord[1],color='black',marker='o')
#     # Customize the plot
#     plt.xlabel('Longitude')
#     plt.ylabel('Latitude')
#     plt.title('Line Plot of Coordinates')
#     plt.grid(True, linestyle='--', alpha=0.7)
#     plt.legend()
#     plt.tight_layout()
#     plt.show()

In [54]:
file_path = 'data.json'

with open(file_path, 'w') as json_file:
    json.dump(nearbytopo[4], json_file, indent=4)